In [2]:
from requests_oauthlib import OAuth1Session
from requests.exceptions import ConnectionError, ReadTimeout, SSLError
from elasticsearch import Elasticsearch
import json, datetime, time, pytz, re, sys,traceback, pymongo
from pymongo import MongoClient
from collections import defaultdict
import numpy as np

In [3]:
KEYS = { # 自分のアカウントで入手したキーを下記に記載
        'consumer_key':'en35U31mW3P9EAqZvR6KXkZ2h',
        'consumer_secret':'JWoHyJU5KRnbeWjb17hVHAQsiPFgwGwvBkZSkRsARrApZqNEKd',
        'access_token':'868744152753647617-ei1ptGYVwJqgv2zy7f09fPVuFrsYbLY',
        'access_secret':'VHQQ1g1MDTUMRufrF2mojkxiig47qEc7ghcGj32fyMX1f',
       }
twitter = None
connect = None
db      = None
tweetdata = None
meta    = None

In [4]:
def initialize(): # twitter接続情報や、mongoDBへの接続処理等initial処理実行
    global twitter, twitter, connect, db, tweetdata, meta
    twitter = OAuth1Session(KEYS['consumer_key'],KEYS['consumer_secret'],
                            KEYS['access_token'],KEYS['access_secret'])
#   connect = Connection('localhost', 27017)     # Connection classは廃止されたのでMongoClientに変更 
    connect = MongoClient('localhost', 27017)
    db = connect.trend
    tweetdata = db.twitter_gekiuma_tweetdata
    meta = db.twitterhash_gekiuma_metadata
initialize()

In [21]:
def getTweetData(search_word, max_id=-1, since_id=-1):
    global twitter
    url = 'https://api.twitter.com/1.1/search/tweets.json'
    params = {'q': search_word,
              'count':'100',
    }
    # max_idの指定があれば設定する
    if max_id != -1:
        params['max_id'] = max_id
    # since_idの指定があれば設定する
    if since_id != -1:
        params['since_id'] = since_id

    req = twitter.get(url, params = params)   # Tweetデータの取得

    # 取得したデータの分解
    if req.status_code == 200: # 成功した場合
        timeline = json.loads(req.text)
        metadata = timeline['search_metadata']
        statuses = timeline['statuses']
        limit = req.headers['x-rate-limit-remaining'] if 'x-rate-limit-remaining' in req.headers else 0
        reset = req.headers['x-rate-limit-reset'] if 'x-rate-limit-reset' in req.headers else 0              
        return {"result":True, "metadata":metadata, "statuses":statuses, "limit":limit, "reset_time":datetime.datetime.fromtimestamp(float(reset)), "reset_time_unix":reset}
    else: # 失敗した場合
        print ("Error: %d" % req.status_code)
        return{"result":False, "status_code":req.status_code}

# 文字列を日本時間2タイムゾーンを合わせた日付型で返す
def str_to_date_jp(str_date):
    dts = datetime.datetime.strptime(str_date,'%a %b %d %H:%M:%S +0000 %Y')
    return pytz.utc.localize(dts).astimezone(pytz.timezone('Asia/Tokyo'))

# 現在時刻をUNIX Timeで返す
def now_unix_time():
    return time.mktime(datetime.datetime.now().timetuple())

In [16]:
#時間指定ありver
def getTweetData(search_word, until, max_id=-1, since_id=-1):
    global twitter
    url = 'https://api.twitter.com/1.1/search/tweets.json'
    params = {'q': search_word,
              'count':'100',
              'until':until
    }
    # max_idの指定があれば設定する
    if max_id != -1:
        params['max_id'] = max_id
    # since_idの指定があれば設定する
    if since_id != -1:
        params['since_id'] = since_id

    req = twitter.get(url, params = params)   # Tweetデータの取得

    # 取得したデータの分解
    if req.status_code == 200: # 成功した場合
        timeline = json.loads(req.text)
        metadata = timeline['search_metadata']
        statuses = timeline['statuses']
        limit = req.headers['x-rate-limit-remaining'] if 'x-rate-limit-remaining' in req.headers else 0
        reset = req.headers['x-rate-limit-reset'] if 'x-rate-limit-reset' in req.headers else 0              
        return {"result":True, "metadata":metadata, "statuses":statuses, "limit":limit, "reset_time":datetime.datetime.fromtimestamp(float(reset)), "reset_time_unix":reset}
    else: # 失敗した場合
        print ("Error: %d" % req.status_code)
        return{"result":False, "status_code":req.status_code}

# 文字列を日本時間2タイムゾーンを合わせた日付型で返す
def str_to_date_jp(str_date):
    dts = datetime.datetime.strptime(str_date,'%a %b %d %H:%M:%S +0000 %Y')
    return pytz.utc.localize(dts).astimezone(pytz.timezone('Asia/Tokyo'))

# 現在時刻をUNIX Timeで返す
def now_unix_time():
    return time.mktime(datetime.datetime.now().timetuple())

In [5]:
a=getTweetData('#ヒルナンデス')

In [23]:
a['statuses'][2]['entities']['hashtags']

[{'indices': [21, 28], 'text': 'ヒルナンデス'}]

In [24]:
a['statuses'][2]['text']

'RT @mcoo41: 唐突にdisるな #ヒルナンデス https://t.co/EsrivWZX39'

In [17]:
def loop_getTweetData(search_word):
    
    sid = -1
    mid = -1 
    count = 0

    res = None
    while(True):    
        try:
            count = count + 1
            sys.stdout.write("%d, "% count)
            res = getTweetData(search_word, max_id=mid, since_id=sid)
            if res['result']==False:
                # 失敗したら終了する
                print("status_code", res['status_code'])
                break

            if int(res['limit']) == 0:    # 回数制限に達したので休憩
                # 日付型の列'created_datetime'を付加する
                print("Adding created_at field.")
                for d in tweetdata.find({'created_datetime':{ "$exists": False }},{'_id':1, 'created_at':1}):
                    #print str_to_date_jp(d['created_at'])
                    tweetdata.update({'_id' : d['_id']}, 
                         {'$set' : {'created_datetime' : str_to_date_jp(d['created_at'])}})
                #remove_duplicates()

                # 待ち時間の計算. リミット＋５秒後に再開する
                diff_sec = int(res['reset_time_unix']) - now_unix_time()
                print("sleep %d sec." % (diff_sec+5))
                if diff_sec > 0:
                    time.sleep(diff_sec + 5)
            else:
                # metadata処理
                if len(res['statuses'])==0:
                    sys.stdout.write("statuses is none. ")
                elif 'next_results' in res['metadata']:
                    # 結果をmongoDBに格納する
                    res.in
                    meta.insert({"metadata":res['metadata'], "insert_date": now_unix_time()})
                    for s in res['statuses']:
                        tweetdata.insert(s)
                    next_url = res['metadata']['next_results']
                    pattern = r".*max_id=([0-9]*)\&.*"
                    ite = re.finditer(pattern, next_url)
                    for i in ite:
                        mid = i.group(1)
                        break
                else:
                    sys.stdout.write("next is none. finished.")
                    break
        except SSLError as E:
            errno, request = E.args
            print("SSLError({0}): {1}".format(errno, strerror))
            print("waiting 5mins")
            time.sleep(5*60)
        except ConnectionError as E:
            print("ConnectionError({0}): {1}".format(errno, strerror))
            print("waiting 5mins")
            time.sleep(5*60)
        except ReadTimeout as E:
            errno, request = E.args
            print("ReadTimeout({0}): {1}".format(errno, strerror))
            print("waiting 5mins")
            time.sleep(5*60)
        except:
            print("Unexpected error:", sys.exc_info()[0])
            traceback.format_exc(sys.exc_info()[2])
            raise
        finally:
            info = sys.exc_info()

SyntaxError: invalid syntax (<ipython-input-17-b29bacae1f46>, line 38)

In [19]:
for i in ite:
                        mid = i.group(1)
                        break

In [21]:
meta

{'_id': ObjectId('5953580d11d8689ab68fa4dd'),
 'insert_date': 1498634253.0,
 'metadata': {'completed_in': 0.198,
  'count': 100,
  'max_id': 879793213921296383,
  'max_id_str': '879793213921296383',
  'next_results': '?max_id=879716198690115588&q=%E6%BF%80%E3%82%A6%E3%83%9E%20until%3A2017-06-28&count=100&include_entities=1',
  'query': '%E6%BF%80%E3%82%A6%E3%83%9E+until%3A2017-06-28',
  'refresh_url': '?since_id=879793213921296383&q=%E6%BF%80%E3%82%A6%E3%83%9E%20until%3A2017-06-28&include_entities=1',
  'since_id': 0,
  'since_id_str': '0'}}

In [16]:
next_url=meta['metadata']['next_results']
pattern = r".*max_id=([0-9]*)\&.*"
ite = re.finditer(pattern, next_url)

In [5]:
getTweetData('めちゃうま')

{'limit': '178',
 'metadata': {'completed_in': 0.046,
  'count': 100,
  'max_id': 880252534316716033,
  'max_id_str': '880252534316716033',
  'next_results': '?max_id=880048621214814208&q=%E3%82%81%E3%81%A1%E3%82%83%E3%81%86%E3%81%BE&count=100&include_entities=1',
  'query': '%E3%82%81%E3%81%A1%E3%82%83%E3%81%86%E3%81%BE',
  'refresh_url': '?since_id=880252534316716033&q=%E3%82%81%E3%81%A1%E3%82%83%E3%81%86%E3%81%BE&include_entities=1',
  'since_id': 0,
  'since_id_str': '0'},
 'reset_time': datetime.datetime(2017, 6, 29, 12, 8, 5),
 'reset_time_unix': '1498705685',
 'result': True,
 'statuses': [{'contributors': None,
   'coordinates': None,
   'created_at': 'Thu Jun 29 02:32:13 +0000 2017',
   'entities': {'hashtags': [],
    'symbols': [],
    'urls': [],
    'user_mentions': []},
   'favorite_count': 3,
   'favorited': False,
   'geo': None,
   'id': 880252534316716033,
   'id_str': '880252534316716033',
   'in_reply_to_screen_name': None,
   'in_reply_to_status_id': None,
   'in_r

In [23]:
    search_word = u'Ｓｉｒａｆｕ'
    sid = -1
    mid = -1 
    count = 0

    res = None
    while(True):    
        try:
            count = count + 1
            sys.stdout.write("%d, "% count)
            res = getTweetData(search_word, max_id=mid, since_id=sid)
            if res['result']==False:
                # 失敗したら終了する
                print("status_code", res['status_code'])
                break

            if int(res['limit']) == 0:    # 回数制限に達したので休憩
                # 日付型の列'created_datetime'を付加する
                print("Adding created_at field.")
                for d in tweetdata.find({'created_datetime':{ "$exists": False }},{'_id':1, 'created_at':1}):
                    #print str_to_date_jp(d['created_at'])
                    tweetdata.update({'_id' : d['_id']}, 
                         {'$set' : {'created_datetime' : str_to_date_jp(d['created_at'])}})
                #remove_duplicates()

                # 待ち時間の計算. リミット＋５秒後に再開する
                diff_sec = int(res['reset_time_unix']) - now_unix_time()
                print("sleep %d sec." % (diff_sec+5))
                if diff_sec > 0:
                    time.sleep(diff_sec + 5)
            else:
                # metadata処理
                if len(res['statuses'])==0:
                    sys.stdout.write("statuses is none. ")
                elif 'next_results' in res['metadata']:
                    # 結果をmongoDBに格納する
                    meta.insert_one({"metadata":res['metadata'], "insert_date": now_unix_time()})
                    for s in res['statuses']:
                        tweetdata.insert(s)
                    next_url = res['metadata']['next_results']
                    pattern = r".*max_id=([0-9]*)\&.*"
                    ite = re.finditer(pattern, next_url)
                    for i in ite:
                        mid = i.group(1)
                        break
                else:
                    for s in res['statuses']:
                        tweetdata.insert(s)
                    sys.stdout.write("next is none. finished.")
                    break
        except SSLError as E:
            errno, request = E.args
            print("SSLError({0}): {1}".format(errno, strerror))
            print("waiting 5mins")
            time.sleep(5*60)
        except ConnectionError as E:
            print("ConnectionError({0}): {1}".format(errno, strerror))
            print("waiting 5mins")
            time.sleep(5*60)
        except ReadTimeout as E:
            errno, request = E.args
            print("ReadTimeout({0}): {1}".format(errno, strerror))
            print("waiting 5mins")
            time.sleep(5*60)
        except:
            print("Unexpected error:", sys.exc_info()[0])
            traceback.format_exc(sys.exc_info()[2])
            raise
        finally:
            info = sys.exc_info()

1, 

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


2, next is none. finished.

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [25]:
    #elasticsearchバージョン
    search_word = u'料理'
    es= Elasticsearch()
    sid = -1
    mid = -1 
    count = 0

    res = None
    while(True):    
        try:
            count = count + 1
            sys.stdout.write("%d, "% count)
            res = getTweetData(search_word, max_id=mid, since_id=sid)
            if res['result']==False:
                # 失敗したら終了する
                print("status_code", res['status_code'])
                break

            if int(res['limit']) == 0:    # 回数制限に達したので休憩
                # 待ち時間の計算. リミット＋５秒後に再開する
                diff_sec = int(res['reset_time_unix']) - now_unix_time()
                print("sleep %d sec." % (diff_sec+5))
                if diff_sec > 0:
                    time.sleep(diff_sec + 5)
            else:
                # metadata処理
                if len(res['statuses'])==0:
                    sys.stdout.write("statuses is none. ")
                elif 'next_results' in res['metadata']:
                    # 結果をmongoDBに格納する
                    doc = res['metadata']
                    doc.update({"insert_date": now_unix_time()})
                    es.index(index="twitter_recipe",doc_type='metadata',body=doc)
                    for s in res['statuses']:
                        es.index(index="twitter_recipe",doc_type='tweetdata',body=s)
                    next_url = res['metadata']['next_results']
                    pattern = r".*max_id=([0-9]*)\&.*"
                    ite = re.finditer(pattern, next_url)
                    for i in ite:
                        mid = i.group(1)
                        break
                else:
                    sys.stdout.write("next is none. finished.")
                    break
        except SSLError as E:
            errno, request = E.args
            print("SSLError({0}): {1}".format(errno, strerror))
            print("waiting 5mins")
            time.sleep(5*60)
        except ConnectionError as E:
            print("ConnectionError({0}): {1}".format(errno, strerror))
            print("waiting 5mins")
            time.sleep(5*60)
        except ReadTimeout as E:
            errno, request = E.args
            print("ReadTimeout({0}): {1}".format(errno, strerror))
            print("waiting 5mins")
            time.sleep(5*60)
        except:
            print("Unexpected error:", sys.exc_info()[0])
            traceback.format_exc(sys.exc_info()[2])
            raise
        finally:
            info = sys.exc_info()

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 

POST http://localhost:9200/twitter_recipe/tweetdata [status:400 request:0.508s]


Unexpected error: <class 'elasticsearch.exceptions.RequestError'>


TypeError: '>=' not supported between instances of 'traceback' and 'int'